# IBM DATA SCIENCE CAPSTONE PROJECT COURSERA

<b>Week 3 Part 1</b>

<b>1.Importing Libraries</b>

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json #library to handle JSON files
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values
import requests #library to handle requests
from bs4 import BeautifulSoup #library to parse HTML and XML documents
from pandas.io.json import json_normalize #tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print("Libraries imported.")

Libraries imported.


<b>2.Scraping data</b>

In [2]:
#GET request
wiki_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup=BeautifulSoup(wiki_data, 'html.parser')

In [4]:
postalCodeList =[]
boroughList =[]
neighborhoodList =[]
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [5]:
df_toronto=pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<b>3.Drop cells with a borough that is "Not assigned"</b>

In [6]:
# drop cells with a borough that is Not assigned
df_toronto_dropna=df_toronto[df_toronto.Borough!= "Not assigned"].reset_index(drop=True)
df_toronto_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<b>4.Grouping Neighbourhoods</b>

In [7]:
df_toronto_grouped=df_toronto_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>5.Neighborhood="Not assigned"</b>

In [8]:
#for Neighborhood="Not assigned", make the value the same as Borough
for index, row in df_toronto_grouped.iterrows():
    if row["Neighborhood"]=="Not assigned":
        row["Neighborhood"]=row["Borough"]       
df_toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>6.Number of rows</b>

In [9]:
df_toronto_grouped.shape

(103, 3)

<b>7.Loading coordinates</b>

In [10]:
#loading the coordinates from the csv file
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

<b>8.Merge tables</b>

In [12]:
# merge two table on the column "PostalCode"
df_toronto_new = df_toronto_grouped.merge(coordinates,on="PostalCode",how="left")
df_toronto_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df_toronto_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<b>9.Get the latitude and longitude values of Toronto</b>

In [14]:
address='Toronto'
geolocator=Nominatim(user_agent="user-toronto")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


<b>10.Create a map of Toronto</b> 

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto=folium.Map(location=[latitude, longitude],zoom_start=10)
# add markers to map
for lat,lng,borough,neighborhood in zip(df_toronto_new['Latitude'],df_toronto_new['Longitude'],df_toronto_new['Borough'],df_toronto_new['Neighborhood']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)      
map_toronto

In [16]:
# filter borough names that contain the word Toronto
borough_names=list(df_toronto_new.Borough.unique())
borough_with_toronto = []
for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [17]:
# create a new DataFrame with only boroughs that contain the word Toronto
df_toronto_new = df_toronto_new[df_toronto_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
df_toronto_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto=folium.Map(location=[latitude, longitude],zoom_start=10)
# add markers to map
for lat,lng,borough,neighborhood in zip(df_toronto_new['Latitude'],df_toronto_new['Longitude'],df_toronto_new['Borough'],df_toronto_new['Neighborhood']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)      
map_toronto

<b>11.Using the Foursquare API to explore the neighborhoods</b>

In [19]:
CLIENT_ID = 'JYXIXCOIJQIMBIFG5ORY3N1OHK51UYMKS42L31HOMWITIL4D'
CLIENT_SECRET = 'E3NMAX2SDHFV4YQNHASOCRE2WRPCZJBBWHKWHDUXGBELD5EN'
VERSION = '20180605'
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET:'+CLIENT_SECRET)

CLIENT_ID: JYXIXCOIJQIMBIFG5ORY3N1OHK51UYMKS42L31HOMWITIL4D
CLIENT_SECRET:E3NMAX2SDHFV4YQNHASOCRE2WRPCZJBBWHKWHDUXGBELD5EN


In [20]:
radius=500
LIMIT=100
venues=[]
for lat,long,post,borough,neighborhood in zip(df_toronto_new['Latitude'],df_toronto_new['Longitude'],df_toronto_new['PostalCode'],df_toronto_new['Borough'],df_toronto_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    results=requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
venues_df=pd.DataFrame(venues)
venues_df.columns=['PostalCode','Borough','Neighborhood','BoroughLatitude','BoroughLongitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory']
venues_df.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [22]:
#Number of venues for each PostalCode
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,11,11,11,11,11,11
M4R,Central Toronto,"North Toronto West, Lawrence Park",19,19,19,19,19,19
M4S,Central Toronto,Davisville,34,34,34,34,34,34
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df['VenueCategory'].unique()[:50]

There are 225 uniques categories.


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Ice Cream Shop', 'Greek Restaurant',
       'Italian Restaurant', 'Brewery', 'Juice Bar',
       'Fruit & Vegetable Store', 'Yoga Studio', 'Restaurant',
       'Pizza Place', 'Bookstore', 'Dessert Shop',
       'Furniture / Home Store', 'Spa', 'Bubble Tea Shop', 'Café',
       'Grocery Store', 'Bakery', 'Coffee Shop', 'Caribbean Restaurant',
       'Indian Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'Sushi Restaurant', 'Liquor Store', 'American Restaurant', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Park',
       'Burrito Place', 'Pet Store', 'Steakhouse', 'Movie Theater',
       'Sandwich Place', 'Food & Drink Shop', 'Fish Market', 'Gay Bar',
       'Cheese Shop', 'Seafood Restaurant', 'Middle Eastern Restaurant',
       'Stationery Store', 'Comfort Food Restaurant', 'Thai Restaurant',
       'Coworking Space', 'Wine Bar'], dtype=object)

In [24]:
toronto_onehot=pd.get_dummies(venues_df[['VenueCategory']],prefix="",prefix_sep="")
toronto_onehot['PostalCode']=venues_df['PostalCode'] 
toronto_onehot['Borough']=venues_df['Borough'] 
toronto_onehot['Neighborhoods']=venues_df['Neighborhood'] 
fixed_columns=list(toronto_onehot.columns[-3:])+list(toronto_onehot.columns[:-3])
toronto_onehot=toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1624, 228)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [25]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
toronto_grouped

,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000

<b>Creating the new dataframe and display the top 10 venues for each PostalCode</b>

In [26]:
num_venues=10
indicators=['st', 'nd', 'rd']

areaColumns=['PostalCode', 'Borough', 'Neighborhoods']
freqColumns=[]
for ind in np.arange(num_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns=areaColumns+freqColumns
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode']=toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough']=toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods']=toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories=toronto_grouped.iloc[ind,:].iloc[3:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:]=row_categories_sorted.index.values[0:num_venues]

neighborhoods_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Neighborhood,Health Food Store,Pub,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Pub,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Steakhouse,Fish & Chips Shop,Sushi Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
5,M4P,Central Toronto,Davisville North,Hotel,Breakfast Spot,Park,Gym,Gym / Fitness Center,Pizza Place,Department Store,Food & Drink Shop,Dance Studio,Sandwich Place
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Restaurant,Rental Car Location,Fast Food Restaurant,Ice Cream Shop,Salon / Barbershop,Mexican Restaurant
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Café,Coffee Shop,Gym,Italian Restaurant,Sushi Restaurant,Park,Diner
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Playground,Trail,Tennis Court,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Light Rail Station,Bank,Sushi Restaurant,Bagel Shop,Restaurant,Sports Bar,Fried Chicken Joint,Pizza Place


<b>12.Cluster Areas</b>

Using k-means to cluster the Toronto areas into 5 clusters

In [27]:
kclusters=5
toronto_grouped_clustering=toronto_grouped.drop(["PostalCode","Borough","Neighborhoods"],1)
#k-means clustering
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 3, 1, 1, 1, 3, 1])

In [28]:
toronto_cp=df_toronto_new.copy()
toronto_cp["Cluster Labels"]=kmeans.labels_
toronto_cp=toronto_cp.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"),on="PostalCode")
toronto_cp.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Health Food Store,Pub,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Pub,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Steakhouse,Fish & Chips Shop,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [29]:
toronto_cp.sort_values(["Cluster Labels"], inplace=True)
toronto_cp

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Health Food Store,Pub,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Yoga Studio
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,1,Coffee Shop,Hotel,Café,Salad Place,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Sushi Restaurant
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Café,Sandwich Place,Coffee Shop,Park,BBQ Joint,Middle Eastern Restaurant,Liquor Store,Pub,History Museum,Burger Joint
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Bakery,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bookstore,Chinese Restaurant,Pub,Dessert Shop
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Grocery Store,Pizza Place,Bar,Park
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Airport Terminal,Airport,Sculpture Garden,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina,Boutique
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,1,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Seafood Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Pub
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Salad Place,Steakhouse,American Restaurant,Asian Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Candy Store,Athletics & Sports,Restaurant,Baby Store,Coffee Shop,Nightclub,Diner


<b>13.Examining the Clusters</b>

In [30]:
toronto_cp.loc[toronto_cp['Cluster Labels']==0,toronto_cp.columns[[1]+list(range(5,toronto_cp.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Neighborhood,Health Food Store,Pub,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Yoga Studio


In [31]:
toronto_cp.loc[toronto_cp['Cluster Labels']==1,toronto_cp.columns[[1]+list(range(5,toronto_cp.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Hotel,Café,Salad Place,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Sushi Restaurant
21,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
24,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Park,BBQ Joint,Middle Eastern Restaurant,Liquor Store,Pub,History Museum,Burger Joint
25,Downtown Toronto,1,Café,Bakery,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bookstore,Chinese Restaurant,Pub,Dessert Shop
26,Downtown Toronto,1,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Grocery Store,Pizza Place,Bar,Park
27,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Airport,Sculpture Garden,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina,Boutique
28,Downtown Toronto,1,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Seafood Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Pub
29,Downtown Toronto,1,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Salad Place,Steakhouse,American Restaurant,Asian Restaurant
30,Downtown Toronto,1,Grocery Store,Café,Park,Candy Store,Athletics & Sports,Restaurant,Baby Store,Coffee Shop,Nightclub,Diner
31,West Toronto,1,Bakery,Pharmacy,Grocery Store,Supermarket,Bank,Bar,Café,Furniture / Home Store,Music Venue,Brewery


In [32]:
toronto_cp.loc[toronto_cp['Cluster Labels']==2,toronto_cp.columns[[1]+list(range(5,toronto_cp.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [33]:
toronto_cp.loc[toronto_cp['Cluster Labels']==3,toronto_cp.columns[[1]+list(range(5,toronto_cp.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Park,Playground,Trail,Tennis Court,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,Central Toronto,3,Park,Swim School,Bus Line,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
10,Downtown Toronto,3,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [34]:
toronto_cp.loc[toronto_cp['Cluster Labels']==4,toronto_cp.columns[[1]+list(range(5,toronto_cp.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
